<a href="https://colab.research.google.com/github/DhrubaAdhikary/GEN_AI_DEMO/blob/master/FineTUned_LLM_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -qU \
#     transformers \
#     datasets \
#     peft \
#     accelerate \
#     bitsandbytes \
#     trl \
#     sentencepiece

In [ ]:
import transformers
import bitsandbytes as bnb

print("Transformers:", transformers.__version__)
print("BitsAndBytes:", bnb.__version__)


Transformers: 5.1.0
BitsAndBytes: 0.49.1


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# ===============================
# Configuration
# ===============================

MODEL_NAME = "tiiuae/falcon-rw-1b"
CHECKPOINT_PATH = "/content/drive/MyDrive/qlora-shakespeare/final_adapter"

# ===============================
# Load 4-bit Base Model
# ===============================

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

# ===============================
# Load LoRA Adapter
# ===============================

model = PeftModel.from_pretrained(base_model, CHECKPOINT_PATH)

model.eval()

print("Checkpoint loaded successfully.")


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/292 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie transformer.word_embeddings.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

ValueError: Can't find 'adapter_config.json' at '/content/drive/MyDrive/qlora-shakespeare/final_adapter'

In [ ]:
def generate_text(prompt,
                  max_new_tokens=150,
                  temperature=0.8,
                  top_p=0.9):

    inputs = tokenizer(prompt, return_tensors="pt")#.to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
prompt = "To be, or not to be,"
result = generate_text(prompt)

print("=== GENERATED TEXT ===\n")
print(result)
